In [1]:
import pandas as pd
import os
from eunjeon import Mecab
from ast import literal_eval

xname = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#8_clean_text'}.xlsx")
df = pd.read_excel(xname, index_col=0)
mecab = Mecab()
corpus = ''
for i in range(len(df)):
    text = literal_eval(df.loc[i, 'text'])
    corpus = corpus + ( "\n".join([" ".join([f for f in mecab.nouns(e) if len(f)>1]) for e in text]))

with open('corpus.txt', 'w',encoding='utf-8') as f:
    for c in corpus:
        f.write("%s" % c)

In [2]:
import logging
from bs4 import BeautifulSoup
from konlpy.tag import Twitter
from gensim.models import word2vec
from pathlib import Path
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


data = word2vec.LineSentence('corpus.txt')
model = word2vec.Word2Vec(size=128, window=2, hs=1, min_count=2, sg=1)
model.build_vocab(data)

for epoch in range(1):

    model.train(data, total_examples=model.corpus_count, epochs=model.iter)

    model.alpha -= 0.002 # decrease the learning rate

    model.min_alpha = model.alpha # fix the learning rate, no decay

model.wv.save_word2vec_format('w2v_0825.txt', binary=False),
model.save("w2v_0825.model")
print("ok")

2020-08-26 03:25:37,206 : INFO : collecting all words and their counts
2020-08-26 03:25:37,212 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-08-26 03:25:37,256 : INFO : PROGRESS: at sentence #10000, processed 118099 words, keeping 8906 word types
2020-08-26 03:25:37,300 : INFO : PROGRESS: at sentence #20000, processed 242590 words, keeping 12354 word types
2020-08-26 03:25:37,346 : INFO : PROGRESS: at sentence #30000, processed 365400 words, keeping 15134 word types
2020-08-26 03:25:37,390 : INFO : PROGRESS: at sentence #40000, processed 478567 words, keeping 17273 word types
2020-08-26 03:25:37,438 : INFO : PROGRESS: at sentence #50000, processed 596532 words, keeping 19428 word types
2020-08-26 03:25:37,481 : INFO : PROGRESS: at sentence #60000, processed 715347 words, keeping 21294 word types
2020-08-26 03:25:37,523 : INFO : PROGRESS: at sentence #70000, processed 833097 words, keeping 22942 word types
2020-08-26 03:25:37,563 : INFO : PROGRESS: at s

2020-08-26 03:26:10,427 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-26 03:26:10,427 : INFO : EPOCH - 5 : training on 2319101 raw words (2286537 effective words) took 5.9s, 387460 effective words/s
2020-08-26 03:26:10,428 : INFO : training on a 11595505 raw words (11432353 effective words) took 27.8s, 410776 effective words/s
2020-08-26 03:26:10,429 : INFO : storing 25247x128 projection weights into w2v_0825.txt
2020-08-26 03:26:12,669 : INFO : saving Word2Vec object under w2v_0825.model, separately None
2020-08-26 03:26:12,670 : INFO : not storing attribute vectors_norm
2020-08-26 03:26:12,670 : INFO : not storing attribute cum_table
2020-08-26 03:26:13,164 : INFO : saved w2v_0825.model


ok


In [3]:
model = word2vec.Word2Vec.load("w2v_0825.model")
d = model.wv.most_similar(positive=["환자"])
for (x, y) in d:
    print("%s - %f" % (x,y))

2020-08-26 03:26:13,183 : INFO : loading Word2Vec object from w2v_0825.model
2020-08-26 03:26:13,539 : INFO : loading wv recursively from w2v_0825.model.wv.* with mmap=None
2020-08-26 03:26:13,541 : INFO : setting ignored attribute vectors_norm to None
2020-08-26 03:26:13,541 : INFO : loading vocabulary recursively from w2v_0825.model.vocabulary.* with mmap=None
2020-08-26 03:26:13,542 : INFO : loading trainables recursively from w2v_0825.model.trainables.* with mmap=None
2020-08-26 03:26:13,542 : INFO : setting ignored attribute cum_table to None
2020-08-26 03:26:13,543 : INFO : loaded w2v_0825.model
2020-08-26 03:26:13,623 : INFO : precomputing L2-norms of word weight vectors


진료 - 0.761073
진찰 - 0.742590
문진 - 0.742022
오더 - 0.728113
입원 - 0.722342
병원 - 0.696627
응급 - 0.695246
내원 - 0.689098
결핵 - 0.689027
소아과 - 0.681651
